<a href="https://colab.research.google.com/github/JunyuYan/Pytorch-Learning-Materials/blob/main/pytorch_official/Optimizing_Model_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters, and optimizes these parameters using gradient descent.

## Hyperparameters

Hyperparameters are adjustable parameters to control the model optimization process. Different hyperparameter values can impact model training and convergence rate.

Genrally, we need to define the following hyperparameters:


*   Number of Epochs: the number times to iterate over datasets
*   Batch size: the number of data samples propogated through the network before the parameters are updated
*   Learning Rate: how much to update model parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training


Each iteration of an optimization loop is called an epoch.
Each epoch consists of two parts:

*   The train loop: iterate over the training dataset to make the model converge to optimal parameters
*   The validate/test loop: iterate over the test dataset to check if the model's performance is improving


## Loss Function
Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Detailed description of different loss functions in pytorch is listed [here](https://pytorch.org/docs/stable/nn.html#loss-functions).

## Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed.

Pytorch supports a number of optimizers, as listed [here](https://pytorch.org/docs/stable/optim.html).

Inside the training loop, optimization happens in three steps:

* Call ``optimizer.zero_grad()`` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
* Backpropagate the prediction loss with a call to ``loss.backward()``. PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call ``optimizer.step()`` to adjust the parameters by the gradients collected in the backward pass.

In the optimizer, we should initialize it with the ``model.parameters`` and ``learning_rate``.

## Full Implementation

Now we can build a complete neural network.

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download datasets
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# Define neural network structure
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()
print(f"Model structure: {model}")

# Deine train and test loop
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  # Set the model to training mode
  model.train()

  for batch, (x, y) in enumerate(dataloader):
    pred = model(x)
    loss = loss_fn(pred, y)

    # backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)*len(x)
      print(f"loss: {loss:>7f} | current: {current:>5d}/{size:>5d}")

def test_loop(dataloader, model, loss_fn):
  # Set the model to evaluatio mode
  model.eval()

  size = len(dataloader.dataset)
  num_batch = len(dataloader)
  total_loss, correct = 0, 0

  # Disable the gradient tracking to avoid unnecessary gradient computations
  with torch.no_grad():
    for x, y in dataloader:
      pred = model(x)
      total_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss = total_loss/num_batch
  correct = correct/size
  print(f"Accuracy: {100*correct:>0.1f}, Average loss: {test_loss:>5f}\n")

# Main function for calling train and test process
epoch = 5
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

for i in range(epoch):
  print(f"Epoch {i+1}:-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!")



Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1:-------------------------------
loss: 2.309927 | current:    64/60000
loss: 2.293953 | current:  6464/60000
loss: 2.287560 | current: 12864/60000
loss: 2.278072 | current: 19264/60000
loss: 2.259205 | current: 25664/60000
loss: 2.232706 | current: 32064/60000
loss: 2.238160 | current: 38464/60000
loss: 2.212430 | current: 44864/60000
loss: 2.208212 | current: 51264/60000
loss: 2.174523 | current: 57664/60000
Accuracy: 45.8, Average loss: 2.175831

Epoch 2:-------------------------------
loss: 2.187833 | current:    64/60000
loss: 2.178426 | current:  6464/60000
loss: 2.139818 | current: 12864/60000
loss: 2.146816 | current: 19264/60000
loss: 2.1044